# Apps Data Analysis

In this project we are going to analyze apps data from Apple store and Google Play so as to reccomend our developer team to develop an app based on our finding. noted that our company develop a free and English language apps, so how the company makes money largely due to ads placment and number of users who use the apps.

<h3>Open and Read the files</h3>

First we will start with opeing our datasets and then convert tham into a list of list so they can be easy to manipulte. After taht we will separate header from rows, so it can be easy  to analyze as well.

below is the datasets we use:
- <a href="https://www.kaggle.com/calibr3io/applestore">Apple store</a>
- <a href="https://www.kaggle.com/somertonman/sample-google-play-for-apps-analysis">Google play</a>

In [ ]:
from csv import reader

opened_file = open('../input/applestore/AppleStore.csv')
read_file = reader(opened_file)
ios = list(read_file)
header_ios = ios[0]
rows_ios = ios[1:]

opened_file = open('../input/sample-google-play-for-apps-analysis/googleplaystore.csv')
read_file = reader(opened_file)
android = list(read_file)
header_android = android[0]
rows_android = android[1:]

We then create a function `explore_data` to explore data from the two datasets.

In [ ]:
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]
    for row in dataset_slice:
        print(row)
        print('\n')
        
    if rows_and_columns:
            print('Number of rows :', len(dataset[:]))
            print('Number of columns :', len(dataset[0]))



We check the header and the rows for each dataset.
As we can see, Apple store dataset has 7197 rows and 16 column and android dataset has 10841 rows and 13 column.

In [ ]:
print(header_ios)
print('\n')
explore_data(rows_ios, 0, 5, True)
print('\n')
print('\n')
print(header_android)
print('\n')
explore_data(rows_android, 0, 5, True)

# Data Cleaning

In this process we will clean the data by removing duplicated rows and error data as well as irrelevent data

<h3>Removing Errors</h3>

Checking if the number of columns is correct for every row

In [ ]:
for index, row in enumerate(rows_android):
    if len(row) != len(header_android):
        print(index,row)

After finding out that there is a row `10472` that has number of columns that is not equal to the header column. we decided to delete that.

In [ ]:
print(rows_android[10472]) #just to recheck the row index is correct


In [ ]:
del rows_android[10472]

print(rows_android[10472])

Then we will find the duplicated rows

In [ ]:
duplicated_apps = []
unique_apps = []

for row in rows_android:
    name = row[0]
    if name in unique_apps:
        duplicated_apps.append(name)
    else:
        unique_apps.append(name)
    

In [ ]:
print(duplicated_apps[:9])

Check pattern for duplicated rows, so we do not remove the duplicated data randomly.

In [ ]:
for row in rows_android:
    app_name = row[0]
    if app_name == 'Instagram':
        print(rows_android.index(row), row)

As we can see that there are 4 duplicated entries here for 'Instagram app.' In column 4, reviews total are different for some data points, so we decided to keep thr one with the highest number of reviews and discard others

we will create a dictionary where each key is a unique app name and the corresponding dictionary value is the highest number of reviews of that app.

In [ ]:
reviews_max = {}

for row in rows_android:
    name = row[0]
    n_reviews = float(row[3])
    if name in reviews_max and reviews_max[name] < n_reviews:
        reviews_max[name] = n_reviews
    elif name not in reviews_max:
        reviews_max[name] = n_reviews
        
        
print(len(reviews_max))

We will use the values from `reviews_max` to delete the duplicated rows

In [ ]:
android_clean = []
already_added = []

for row in rows_android:
    name = row[0]
    n_reviews = float(row[3])
    if n_reviews == reviews_max[name] and name not in already_added:
        android_clean.append(row)
        already_added.append(name)
        
        
explore_data(android_clean,0,3,True)

Here we will delete the non-English apps out, since our company develop apps towards English audience

We start by craeting a function to check if all the strings of a word is in English.

In [ ]:
def is_english(string_name):
    for cha in string_name:
        order = ord(cha) # we use a corresponding number associated with a character. For an English character, it is from 0 to 127
        if order > 127:
            return False
    return True
        
print(is_english('爱奇艺PPS -《欢乐颂2》电视剧热播'))
print(is_english('Docs To Go™ Free Office Suite'))
print(is_english('Instachat 😜'))
print(is_english('Instagram'))

However some characters such as 😜, and ™ are also outside the 127 range and it return `False` although the app is in English. To minimize the impact of data loss, we'll only remove an app if its name has more than three characters with corresponding numbers falling outside the ASCII range.

In [ ]:
def is_english(string_name):
    non_ascii = 0
    
    for cha in string_name:
        order = ord(cha) # we use a corresponding number associated with a character. For an English character, it is from 0 to 127
        if order > 127:
            non_ascii += 1
            
    if non_ascii > 3:
        return False
    else:
        return True
        
print(is_english('爱奇艺PPS -《欢乐颂2》电视剧热播'))
print(is_english('Docs To Go™ Free Office Suite'))
print(is_english('Instachat 😜'))
print(is_english('Instagram'))

In [ ]:
android_englsih_apps = []
ios_englsih_apps = []

for row in android_clean:
    name = row[0]
    english_app = is_english(name)
    
    if english_app == True:
        android_englsih_apps.append(row)
        
for row in rows_ios:
    name = row[1]
    english_app = is_english(name)
    
    if english_app == True:
        ios_englsih_apps.append(row)
        
explore_data(android_englsih_apps,0,7,True)
print('\n')
explore_data(ios_englsih_apps,0,7,True)
    
    

Check the header again to see the price column, since our team develop a free app.

In [ ]:
print(header_ios)
print('\n')
print(ios_englsih_apps[:7])
print('\n')
print(header_android)
print('\n')
print(android_englsih_apps[:7])
print('\n')


While we use loop to get all the free app into lists, we got a problem with Google play dataset since it contains `'$'` so we need to remove it first before we can convert them into flaot.

In [ ]:
free_ios = []
free_android = []

for index, row in enumerate(android_englsih_apps):
    prices = row[7]
    for cha in prices:
        if cha == '$':
            row[7] = prices.replace('$','')

            
for row in android_englsih_apps:
    price = float(row[7])
    if price == 0:
        free_android.append(row)
        
        
for index, row in enumerate(ios_englsih_apps):
    prices = row[4]
    for cha in prices:
        if cha == '$':
            row[7] = prices.replace('$','')

            
for row in ios_englsih_apps:
    price = float(row[4])
    if price == 0:
        free_ios.append(row)
        
explore_data(free_android,0,7,True)
print('\n')
print('\n')
explore_data(free_ios,0,7,True)

# Analysis 

As we mentioned in the introduction, our aim is to determine the kinds of apps that are likely to attract more users because our revenue is highly influenced by the number of people using our apps.

To minimize risks and overhead, our validation strategy for an app idea is comprised of three steps:

-Build a minimal Android version of the app, and add it to Google Play.

-If the app has a good response from users, we develop it further.

-If the app is profitable after six months, we build an iOS version of the app and add it to the App Store.

Because our end goal is to add the app on both Google Play and the App Store, we need to find app profiles that are successful on both markets.

First, let's which genre is the most common in the market. Here, we will use frequncy table and then we will sort them ordered by their genres' share.

In [ ]:
def freq_count(a_list, index):
    table = {}
    for row in a_list:
            name = row[index]
            if name in table:
                table[name] += 1
            else:
                table[name] = 1
    return table

def freq_sum(a_dict):
    sum_freq = 0
    for element in a_dict:
        sum_freq += a_dict[element]
    return sum_freq
        
        
def freq_table(dataset, index):
        freq_table = {}
        table_dict = freq_count(dataset, index)
        sum_table = freq_sum(table_dict)
        for element in table_dict:
            freq_table[element] = table_dict[element] * 100 / sum_table
        return freq_table
        

def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])


We use the function to print out percentages of each categories to see the shares of genres in the market

In [ ]:
print('prime_genre')        
display_table(free_ios,11)

After analyzing frequncy tables from ios prime_genres, Google play genres, and Google play categories, our conclusions are as follows:

The most common prime genres in ios free and English apps is Games and the second most common is Entertainment. However this is far less from 58% of Games genre. We also see that apps related to entertainment such as Games, Entertainment, and Photo & Video are placed high in the ranks.
    

    


In [ ]:
print('genres') 
display_table(free_android,9)

The difference between the Genres and the Category columns is not crystal clear, but one thing we can notice is that the Genres column is much more granular (it has more categories). We're only looking for the bigger picture at the moment, so we will not use genres for Google Play store here since it is more ganular level.

In [ ]:
print('category') 
display_table(free_android,1)

According to the result of category, we see that the most popular genres is Family which is mostly kids game. As such we can assume that the genres that have the high market share from both Apple Store and Google Play are related to fun.

<h3>Most popular app in app store</h3>

After seeing which genres are the most common, we then furthur analyze to see which genres is the most popular, since high market share does not mean high rating, or people like it. For the Google Play data set, we can find this information in the Installs column, but for the App Store data set this information is missing. As a workaround, we'll take the total number of user ratings as a proxy, which we can find in the rating_count_tot app.



In [ ]:
ios_table = freq_table(free_ios,11)
unique_genres = []

for element in ios_table:
    unique_genres.append(element)
    
for genre in unique_genres:
    total = 0
    len_genres = 0
    for app in free_ios:
        genre_app = app[11]
        if genre_app == genre:
            total += float(app[5])
            len_genres +=1
    avg_rating = total / len_genres
    print(genre,':',avg_rating)
    print('\n')

On average, navigation apps have the highest number of user reviews, but this figure is heavily influenced by Waze and Google Maps, which have close to half a million user reviews together:

Here, we will dig a little deeper to see what influence that.

In [ ]:
for row in free_ios:
    if row[-5] == 'Navigation':
        print(row[1],':',row[5])

The same pattern applies to social networking apps, where the average number is heavily influenced by a few giants like Facebook, Pinterest, Skype, etc. Same applies to music apps, where a few big players like Pandora, Spotify, and Shazam heavily influence the average number.

Our aim is to find popular genres, but navigation, social networking or music apps might seem more popular than they really are. The average number of ratings seem to be skewed by very few apps which have hundreds of thousands of user ratings, while the other apps may struggle to get past the 10,000 threshold. We could get a better picture by removing these extremely popular apps for each genre and then rework the averages, but we'll leave this level of detail for later.

Reference apps have 74,942 user ratings on average, but it's actually the Bible and Dictionary.com which skew up the average rating:

In [ ]:
for row in free_ios:
    if row[-5] == 'Reference':
        print(row[1],':',row[5])

In [ ]:
display_table(free_android,5)

However, this niche seems to show some potential. One thing we could do is take another popular book and turn it into an app where we could add different features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes about the book, etc. On top of that, we could also embed a dictionary within the app, so users don't need to exit our app to look up words in an external app.

This idea seems to fit well with the fact that the App Store is dominated by for-fun apps. This suggests the market might be a bit saturated with for-fun apps, which means a practical app might have more of a chance to stand out among the huge number of apps on the App Store.

Other genres that seem popular include weather, book, food and drink, or finance. The book genre seem to overlap a bit with the app idea we described above, but the other genres don't seem too interesting to us:

Weather apps — people generally don't spend too much time in-app, and the chances of making profit from in-app adds are low. Also, getting reliable live weather data may require us to connect our apps to non-free APIs.

Food and drink — examples here include Starbucks, Dunkin' Donuts, McDonald's, etc. So making a popular food and drink app requires actual cooking and a delivery service, which is outside the scope of our company.

Finance apps — these apps involve banking, paying bills, money transfer, etc. Building a finance app requires domain knowledge, and we don't want to hire a finance expert just to build an app.

Now let's analyze the Google Play market a bit.

For the Google Play market, we actually have data about the number of installs, so we should be able to get a clearer picture about genre popularity. However, the install numbers don't seem precise enough — we can see that most values are open-ended (100+, 1,000+, 5,000+, etc.)

In [ ]:
display_table(free_android, 5)

One problem with this data is that is not precise. For instance, we don't know whether an app with 100,000+ installs has 100,000 installs, 200,000, or 350,000. However, we don't need very precise data for our purposes — we only want to get an idea which app genres attract the most users, and we don't need perfect precision with respect to the number of users.

We're going to leave the numbers as they are, which means that we'll consider that an app with 100,000+ installs has 100,000 installs, and an app with 1,000,000+ installs has 1,000,000 installs, and so on.

To perform computations, however, we'll need to convert each install number to float — this means that we need to remove the commas and the plus characters, otherwise the conversion will fail and raise an error. We'll do this directly in the loop below, where we also compute the average number of installs for each genre (category).

In [ ]:
android_table = freq_table(free_android,1)
unique_categories = []

for element in android_table:
    unique_categories.append(element)
    
for category in unique_categories:
    total = 0
    len_cat = 0
    for row in free_android:
        if row[1] == category:
            num = row[5].replace('+','')
            num2 = num.replace(',','')
            total += float(num2)
            len_cat += 1
    avg_rating = total / len_cat
    print(category,':',avg_rating)
    print('\n')
    
            

On average, communication apps have the most installs: 38,456,119. This number is heavily skewed up by a few apps that have over one billion installs (WhatsApp, Facebook Messenger, Skype, Google Chrome, Gmail, and Hangouts), and a few others with over 100 and 500 million installs:

In [ ]:
for row in free_android:
    if row[1] == 'COMMUNICATION' and (row[5] == '100,000,000+' or 
                             row[5] == '1,000,000,000+' or
                             row[5] == '500,000,000+'):
        print(row[0],':',row[5])

In [ ]:
under_100_m = []

for app in free_android:
    n_installs = app[5]
    n_installs = n_installs.replace(',', '')
    n_installs = n_installs.replace('+', '')
    if (app[1] == 'COMMUNICATION') and (float(n_installs) < 100000000):
        under_100_m.append(float(n_installs))
        
sum(under_100_m) / len(under_100_m)

We see the same pattern for the video players category, which is the runner-up with 24,727,872 installs. The market is dominated by apps like Youtube, Google Play Movies & TV, or MX Player. The pattern is repeated for social apps (where we have giants like Facebook, Instagram, Google+, etc.), photography apps (Google Photos and other popular photo editors), or productivity apps (Microsoft Word, Dropbox, Google Calendar, Evernote, etc.).

Again, the main concern is that these app genres might seem more popular than they really are. Moreover, these niches seem to be dominated by a few giants who are hard to compete against.

The game genre seems pretty popular, but previously we found out this part of the market seems a bit saturated, so we'd like to come up with a different app recommendation if possible.

The books and reference genre looks fairly popular as well, with an average number of installs of 8,767,811. It's interesting to explore this in more depth, since we found this genre has some potential to work well on the App Store, and our aim is to recommend an app genre that shows potential for being profitable on both the App Store and Google Play.

Let's take a look at some of the apps from this genre and their number of installs:

In [ ]:
for app in free_android:
    if app[1] == 'BOOKS_AND_REFERENCE':
        print(app[0], ':', app[5])

The book and reference genre includes a variety of apps: software for processing and reading ebooks, various collections of libraries, dictionaries, tutorials on programming or languages, etc. It seems there's still a small number of extremely popular apps that skew the average:

In [ ]:
for app in free_android:
    if app[1] == 'BOOKS_AND_REFERENCE' and (app[5] == '1,000,000+'
                                            or app[5] == '5,000,000+'
                                            or app[5] == '10,000,000+'
                                            or app[5] == '50,000,000+'):
        print(app[0], ':', app[5])

This niche seems to be dominated by software for processing and reading ebooks, as well as various collections of libraries and dictionaries, so it's probably not a good idea to build similar apps since there'll be some significant competition.

We also notice there are quite a few apps built around the book Quran, which suggests that building an app around a popular book can be profitable. It seems that taking a popular book (perhaps a more recent book) and turning it into an app could be profitable for both the Google Play and the App Store markets.

However, it looks like the market is already full of libraries, so we need to add some special features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes on the book, a forum where people can discuss the book, etc.

# Conclusions

In this project, we analyzed data about the App Store and Google Play mobile apps with the goal of recommending an app profile that can be profitable for both markets.

We concluded that taking a popular book (perhaps a more recent book) and turning it into an app could be profitable for both the Google Play and the App Store markets. The markets are already full of libraries, so we need to add some special features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes on the book, a forum where people can discuss the book, etc.